## 1. Load the packages

In [ ]:
# Data processing packages
import numpy as np
import pandas as pd
from collections import Counter

# Machine learning packages
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SequentialFeatureSelector, RFE, SelectPercentile, chi2, mutual_info_regression, SelectFromModel
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

import torch

# Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# Others
import time
from pathlib import Path

## 2. Read the data

In [ ]:
X1 = pd.read_csv("X1.csv")
Y1 = pd.read_csv("Y1.csv", header=None, names=['revenue '])
X2 = pd.read_csv("X2.csv")

In [ ]:
X1

## 3. Data Preprocessing

### STEP 1: Data Cleaning (columns drop off and missing value processing)

In [ ]:
def data_cleaning_process(df):
    """
    This function will drop columns, like "Unnamed: 0", "title", "img_url", "description" from the dataset, and replace the missing value in `runtime` column with a median value, and replace the missing value in `genres` with "Others".
    :param df: A dataframe (X1 or X2)
    :return: A new cleaned dataframe
    """

    new_df = df.copy()
    # missing value for runtime: replace "\\N" with median value
    median_runtime = np.median(new_df.loc[new_df['runtime'] != '\\N', 'runtime'].astype(np.int64))
    new_df['runtime'] = np.where(new_df['runtime'] == '\\N', median_runtime, new_df['runtime']).astype(np.int64)

    # missing value for genres: replace "\\N" with "Others"
    new_df.loc[new_df['genres'] == "\\N", "genres"] = "Others"

    # drop "Unnamed: 0", "title", "img_url", "description"
    new_df = new_df.drop(["Unnamed: 0", "title", "img_url", "description"], axis=1)

    return new_df

In [ ]:
X1_cleaned = data_cleaning_process(X1)

### STEP 2: Data Type Split (Numerical, Categorical, Embeddings)

In [ ]:
def data_type_split(df):
    """
    This function will split the whole dataset into different sub dataset according to the data types of the columns
    :param df: A dataframe
    :return: three datadrames, which are numerical, categorical, embeddings
    """

    new_df = df.copy()
    numeric_features = new_df.select_dtypes(include="number").columns.tolist()
    non_numeric_features = new_df.select_dtypes(exclude="number").columns.tolist()
    embedding_features = ['img_embeddings', 'text_embeddings']
    numeric_features.remove('is_adult')
    categorical_features = non_numeric_features.copy()
    [categorical_features.remove(col) for col in embedding_features]
    categorical_features.append('is_adult')
    return new_df.loc[:, numeric_features], new_df.loc[:, categorical_features], new_df.loc[:, embedding_features]

In [ ]:
df_num, df_cat, df_emb = data_type_split(X1_cleaned)

In [ ]:
df_num.head()

In [ ]:
df_cat.head()

In [ ]:
df_emb.head()

### STEP 3: Categorical Columns Processing (Genres --> multilable binary type, Studio --> studio_frequency)

In [ ]:
dict_cat_freq = torch.load("studio_freq")

In [ ]:
mlb = MultiLabelBinarizer()
df_cat['genres_split'] = df_cat['genres'].apply(lambda x: x.split(","))
mlb.fit(df_cat['genres_split'])

In [ ]:
def categorical_process(df):
    """
    This function will process on `genres` and `studio` columns.
    `genres` will be transformed to multilabel binary variables;
    `studio` will be transformed to a frequency type.
    :param df: A categorical datframe
    :return: A new dataframe
    """

    new_df = df.copy()

    # processing on `genres` column
    new_df['genres_split'] = new_df['genres'].apply(lambda x: x.split(","))
    # mlb = MultiLabelBinarizer()
    genere_encoder_df = pd.DataFrame(mlb.transform(new_df['genres_split']))
    genere_encoder_df.columns = mlb.classes_.tolist()

    # processing on `studio` column
    studio_freq_df = pd.DataFrame(new_df['studio'].apply(lambda x: dict_cat_freq[x] if x in dict_cat_freq.keys() else min(dict_cat_freq.values())))
    studio_freq_df.columns = ['studio_freq']


    processed_cat_df = pd.concat([genere_encoder_df, studio_freq_df, new_df['is_adult']], axis=1)

    return processed_cat_df

In [ ]:
categorical_process(df_cat).columns

### STEP 4: Embedding Column Processing

In [ ]:
def listToDF(df, column_name):

    new_df = []
    for row in df[column_name]:
        ls = []
        row = eval(row)
        for each in row:
            ls.append(each)
        new_df.append(ls)

    new_df = pd.DataFrame(new_df)

    return new_df

In [ ]:
def embedding_process(df):
    new_df = df.copy()

    # image embeddings
    img_emb_df = listToDF(new_df, 'img_embeddings')
    text_emb_df = listToDF(new_df, 'text_embeddings')

    processed_emb_df = pd.concat([img_emb_df, text_emb_df], axis=1)
    return processed_emb_df

In [ ]:
embedding_process(X1.loc[:, ['img_embeddings', 'text_embeddings']])

### STEP 5: Combine Everything

In [ ]:
def data_combine(df_num, df_cat, df_emb):
    new_df = pd.concat([df_num, df_cat, df_emb], axis=1)
    return new_df

In [ ]:
df_processed = data_combine(df_num, categorical_process(df_cat), df_emb)

In [ ]:
df_processed

### STEP 6: Normalization and Standarization

In [ ]:
def data_scaling(df):
    """
    This function will process on the numercial columns.
    For `ratings`, we will use normalization
    For the other columns, we will use standardization
    :param df:
    :return:
    """
    new_X = df.copy().to_numpy()
    # new_df = df.copy()

    # df_norm = new_df["ratings"]
    # df_stad = new_df.iloc[:, 1:]
    scaler_norm = MinMaxScaler().fit(new_X[:, 0].reshape([-1, 1]))
    scaler_stad = StandardScaler().fit(new_X[:, 1:5])
    new_X[:, 0] = scaler_norm.transform(new_X[:, 0].reshape([-1, 1])).ravel()
    new_X[:, 1:5] = scaler_stad.transform(new_X[:, 1:5])

    return new_X

In [ ]:
data_scaling(df_processed)

### Construct Data Engineering Pipeline

In [ ]:
def DataEngineering(df):
    df_cleaned = data_cleaning_process(df)
    df_num, df_cat, df_emb = data_type_split(df_cleaned)
    df_cat_processed = categorical_process(df_cat)
    df_emb_processed = embedding_process(df_emb)
    df_processed = data_combine(df_num, df_cat_processed, df_emb_processed)
    X_ready = data_scaling(df_processed)

    return X_ready

In [ ]:
preprocess_transformer = FunctionTransformer(DataEngineering)

In [ ]:
p1 = Pipeline([
    ('Preprocessor', preprocess_transformer)
])

In [ ]:
X1_pre = p1.fit_transform(X1)
X1_pre[:5, :]

## 4. Feature Selection

### 4.1 Give the column names to the dataset

In [ ]:
def renamingDataset(X):
    new_X = X.copy()
    new_df = pd.DataFrame(new_X)
    num_col_names = ['ratings', 'n_votes', 'production_year', 'runtime', 'release_year']
    cat_col_names = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Others',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western', 'studio_freq', 'is_adult']
    img_emb_names = []
    for i in range(2048):
        img_emb_names.append("img_emb_" + str(i))

    text_emb_names = []
    for i in range(768):
        text_emb_names.append("text_emb_" + str(i))

    all_col_names = num_col_names + cat_col_names + img_emb_names + text_emb_names

    new_df.columns = all_col_names

    return new_df

In [ ]:
renamed_df_ready = renamingDataset(X1_pre)
renamed_df_ready.head()

In [ ]:
def divideDataset(df):
    new_df = df.copy()

    non_emb_df = new_df.iloc[:, :34]
    img_emb_df = new_df.iloc[:, 34:2082]
    text_emb_df = new_df.iloc[:, 2082:]

    return non_emb_df, img_emb_df, text_emb_df # pd.dataframe type

In [ ]:
non_emb_df, img_emb_df, text_emb_df = divideDataset(renamed_df_ready)

### 4.2 DR on Embeddings

In [ ]:
scaler_emb = MinMaxScaler()
emb_df_scaled = scaler_emb.fit_transform(pd.concat([img_emb_df, text_emb_df], axis=1))
pca_emb = PCA(n_components=0.9)
pca_emb.fit(emb_df_scaled)

def DRembeddings(img_emb_df, text_emb_df):

    # scale the data to the range between 0 and 1 before using PCA
    # scaler_emb = MinMaxScaler()
    # img_emb_df_scaled = scaler_emb.fit_transform(img_emb_df)
    # text_emb_df_scaled = scaler_emb.fit_transform(text_emb_df)
    emb_df_scaled = scaler_emb.fit_transform(pd.concat([img_emb_df, text_emb_df], axis=1))

    # pca_emb = PCA(n_components=n_components)
    # df_reduced_emb = pd.DataFrame(pca_emb.fit_transform(emb_df_scaled))
    df_reduced_emb = pd.DataFrame(pca_emb.transform(emb_df_scaled))

    emb_col_names = []
    for i in range(df_reduced_emb.shape[1]):
        emb_col_names.append("emb_" + str(i))

    df_reduced_emb.columns = emb_col_names

    return df_reduced_emb

In [ ]:
df_reduced_emb = DRembeddings(img_emb_df, text_emb_df)
df_reduced_emb.head()

### 4.3 Feature Selection

We will predict the log(1 + Y1)

In [ ]:
"""
    The forward feature selection takes a bit time, so we saved the feature selection reulst into a file using torch
"""
# np.random.seed(42)
# rfr = RandomForestRegressor(n_jobs=-1)
# sfs_34features = SequentialFeatureSelector(
#     rfr,
#     k_features=34,
#     forward=True,
#     floating=False,
#     verbose=2,
#     scoring='r2',
#     cv=10,
# ).fit(non_emb_df, np.log(1 + Y1))
# if Path('../models').exists():
#     torch.save("../models/foward_feature_selection_20features")
# else:
#     Path('../model').mkdir(parents=True, exist_ok=True)
#     torch.save("../models/foward_feature_selection_20features")

In [ ]:
# Load the feature selection result
sfs_34 = torch.load("../models/foward_feature_selection_20features")

In [ ]:
pd.DataFrame.from_dict(sfs_34.get_metric_dict()).T

In [ ]:
fig1 = plot_sfs(sfs_34.get_metric_dict(), kind='std_dev')
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()

In [ ]:
sfs_34.k_score_

In [ ]:
sfs_34.k_feature_idx_

In [ ]:
features_selected = list(sfs_34.k_feature_names_)
non_emb_df_selected = non_emb_df[features_selected]
non_emb_df_selected.head()

### 4.4 Combine selected non_emb with reduced emb

In [ ]:
def combineNonReducedEmb(non_emb_df_selected, reduced_emb_df):
    DR_df = pd.concat([non_emb_df_selected, reduced_emb_df], axis=1)
    return DR_df

In [ ]:
DR_df = combineNonReducedEmb(non_emb_df_selected, df_reduced_emb)
DR_df.head()

### Construct Feature Selection Pipeline

In [ ]:
def FeatureSelection(X):
    renamed_df_ready = renamingDataset(X)
    non_emb_df, img_emb_df, text_emb_df = divideDataset(renamed_df_ready)
    reduced_emb_df = DRembeddings(img_emb_df, text_emb_df)
    non_emb_df_selected = non_emb_df[features_selected]
    DR_df = combineNonReducedEmb(non_emb_df_selected, reduced_emb_df)

    return DR_df

In [ ]:
feature_selection_transformer = FunctionTransformer(FeatureSelection)

In [ ]:
p2 = Pipeline([
    ('Preprocessor', preprocess_transformer),
    ('FeatureSelection', feature_selection_transformer),
])

In [ ]:
X1_ready = p2.fit_transform(X1)
X1_ready.head()